# Convolution Neural Networks with GLOVE o WORD2VEC

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../") 

from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer
)
from sklearn.metrics import f1_score
from ast import literal_eval

from personal_library.sce_keras.callbacks import (
    F1PrecisionRecall,
    LearningRateDecay
)
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1

/Users/sebastiancorrea/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/sebastiancorrea/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
x = np.load('../data/dataset/X.npy')#.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
y = np.load('../data/dataset/y.npy')

shape = x[0].shape
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=1992, test_size=0.2)

In [ ]:
from personal_library.sce_keras.layers.custom_layers import CBGN
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential 
from keras.optimizers import Adam
from keras.layers import (
    Flatten,
    Activation,
    Dense,
    LSTM,
    Dropout,
    SpatialDropout1D
)


num_classes = 1
batch_size = 32
epochs = 20
learnRate = 0.00001

metric = F1PrecisionRecall()
lrate_decay = LearningRateDecay(epochs, learnRate).step_decay

checkpoint_path = "../model_wehigts/conv_wehigts.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=1,
                               save_best_only=True, mode='min')

model = Sequential()
model.add(SpatialDropout1D(0.2, input_shape=shape))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))

adam = Adam(lr=learnRate, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)

# 'binary_crossentropy'
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy', f1]) 

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                   )

Train on 25569 samples, validate on 6393 samples
Epoch 1/20
25569/25569 [==============================] - 34s 1ms/step - loss: 0.2139 - acc: 0.9308 - f1: 0.1960 - val_loss: 0.2071 - val_acc: 0.9379 - val_f1: 0.1894
Epoch 2/20
25569/25569 [==============================] - 28s 1ms/step - loss: 0.1914 - acc: 0.9369 - f1: 0.2418 - val_loss: 0.1864 - val_acc: 0.9356 - val_f1: 0.1863
Epoch 3/20
25569/25569 [==============================] - 34s 1ms/step - loss: 0.1826 - acc: 0.9408 - f1: 0.2685 - val_loss: 0.1718 - val_acc: 0.9440 - val_f1: 0.2533
Epoch 4/20
25569/25569 [==============================] - 31s 1ms/step - loss: 0.1761 - acc: 0.9416 - f1: 0.2820 - val_loss: 0.1740 - val_acc: 0.9454 - val_f1: 0.2796
Epoch 5/20
25569/25569 [==============================] - 32s 1ms/step - loss: 0.1683 - acc: 0.9448 - f1: 0.3059 - val_loss: 0.1753 - val_acc: 0.9465 - val_f1: 0.2683
Epoch 6/20
25569/25569 [==============================] - 30s 1ms/step - loss: 0.1659 - acc: 0.9451 - f1: 0.3216 - v